In [15]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
from tensorflow.keras.layers import Conv2D,MaxPool2D,Activation,Flatten,Dense
from tensorflow.keras import Sequential
import zipfile


In [5]:
import zipfile

# Download data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("10_food_classes_10_percent.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("Testing images:")
test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")

In [7]:
import datetime
def create_tensorboard_callback(dir_name,experiment_name):
    log_dir=dir_name+'/'+experiment_name+'/'+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorflow_callback=tf.keras.callbacks.TensorBoard(
        log_dir=log_dir
    )
    print(f"saving log files : {log_dir}")
    return tensorflow_callback

In [8]:
import tensorflow_hub as hub

In [9]:
resnet_url="https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"
efficent_net_url="https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"


In [10]:
def create_model(model_url,num_classes=10):
    feature_extractor_model=hub.KerasLayer(model_url,trainable=False,name='feature_extraction_layer',input_shape=IMAGE_SHAPE+(3,))
    model=tf.keras.Sequential([feature_extractor_model,tf.keras.layers.Dense(num_classes,activation='softmax',name='output_layer')])
    return model

In [16]:
resnet_model=create_model(resnet_url,train_data_10_percent.num_classes)
resnet_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
resnet_history=resnet_model.fit(train_data_10_percent,
                epochs=5,
                steps_per_epoch=len(train_data_10_percent),
                validation_data=test_data,
                validation_steps=len(test_data),
                callbacks=[create_tensorboard_callback(dir_name='tensoflow_hub',experiment_name='resnet50v2')])
                

In [17]:
def plot_loss_curves(history):
    loss=history.history['loss']
    val_loss=history.history['val_loss']
    
    accuracy=history.history['accuracy']
    val_accuracy=history.history['val_accuracy']
    
    epochs=range(len(history.history['loss']))
    
    plt.plot(epochs, loss, label='training_loss')
    plt.plot(epochs, val_loss, label='val_loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.legend()
    
    plt.figure()
    plt.plot(epochs, accuracy, label='training_accuracy')
    plt.plot(epochs, val_accuracy, label='val_accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.legend();

In [18]:
plot_loss_curves(resnet_history)

In [19]:
resnet_model.summary()

In [20]:
resnet_model.evaluate(test_data)

In [21]:
efficient_net_model=create_model(efficent_net_url,train_data_10_percent.num_classes)
efficient_net_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
efficient_net_history=efficient_net_model.fit(train_data_10_percent,
                epochs=5,
                steps_per_epoch=len(train_data_10_percent),
                validation_data=test_data,
                validation_steps=len(test_data),
                callbacks=[create_tensorboard_callback(dir_name='tensoflow_hub',experiment_name='efficient_netB0')])
                

In [22]:
efficient_net_model.summary()

In [23]:
plot_loss_curves(efficient_net_history)

In [24]:
efficient_net_model_1=create_model(efficent_net_url,train_data_10_percent.num_classes)
efficient_net_model_1.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
efficient_net_history_1=efficient_net_model_1.fit(train_data_10_percent,
                epochs=20,
                steps_per_epoch=len(train_data_10_percent),
                validation_data=test_data,
                validation_steps=len(test_data),
                callbacks=[create_tensorboard_callback(dir_name='tensoflow_hub',experiment_name='efficient_netB0')])
                

In [25]:
plot_loss_curves(efficient_net_history_1)

In [26]:
efficient_net_model_1.summary()

In [27]:
mobile_net_url='https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

In [28]:
tf.random.set_seed(42)
mobile_net_model=create_model(mobile_net_url,train_data_10_percent.num_classes)
mobile_net_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
                    optimizer=tf.keras.optimizers.Adam(),
                    metrics=['accuracy'])
mobile_net_model_history=mobile_net_model.fit(train_data_10_percent,
                                             callbacks=[create_tensorboard_callback(dir_name='tensorflow_hub',experiment_name='mobilenet')],
                                             validation_data=test_data,
                                             validation_steps=len(test_data),
                                             epochs=5,
                                             steps_per_epoch=len(train_data_10_percent))

In [29]:
plot_loss_curves(mobile_net_model_history)

In [30]:
mobile_net_model.summary()

Lesson 5 
transfer learning more data

In [31]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py 
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, walk_through_dir

In [32]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip 

unzip_data("10_food_classes_10_percent.zip")
walk_through_dir("10_food_classes_10_percent")

In [33]:
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

IMAGE_SHAPE=(224,224)

In [34]:
train_data_10_percent=tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                         image_size=IMAGE_SHAPE,
                                                                         label_mode='categorical',
                                                                         batch_size=32)
test_data_10_percent=tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                        image_size=IMAGE_SHAPE,
                                                                        batch_size=32,
                                                                        label_mode='categorical')


In [35]:
# for img,lab in train_data_10_percent.take(1):
#     print(img,lab)

In [38]:
# plot_loss_curves(history_10_percent)

In [37]:
base_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False
inputs=tf.keras.layers.Input(shape=(224,224,3),name='input_layer')
x=base_model(inputs)
print(x.shape)

x=tf.keras.layers.GlobalAveragePooling2D()(x)

outputs=tf.keras.layers.Dense(10,activation='softmax',name='output_layer')(x)

model_0=tf.keras.Model(inputs,outputs)

model_0.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
history_0=model_0.fit(train_data_10_percent,
           epochs=5,
           validation_data=test_data_10_percent,
           validation_steps=len(test_data_10_percent),
           steps_per_epoch=len(train_data_10_percent),
           callbacks=[create_tensorboard_callback("transfer_learning", "10_percent_feature_extract")])

In [39]:
model_0.summary()

In [40]:
plot_loss_curves(history_0)

In [41]:
input_shape=(1,5,5,3)
tf.random.set_seed(42)
input_tensor=tf.random.normal(input_shape)
print(input_tensor)

global_avg_pooling=tf.keras.layers.GlobalAveragePooling2D()(input_tensor)

print(global_avg_pooling)

print(global_avg_pooling.shape)

print(input_tensor.shape)

In [42]:
tf.reduce_mean(input_tensor,axis=[1,2])

In [43]:
base_model_max=tf.keras.applications.EfficientNetB0(include_top=False)
base_model_max.trainable=False

inputs=tf.keras.layers.Input(shape=(224,224,3),name='input_layer')
x=base_model_max(inputs)

x=tf.keras.layers.GlobalMaxPool2D()(x)
outputs=tf.keras.layers.Dense(10,activation='sigmoid',name='output')(x)

model_0_max=tf.keras.Model(inputs,outputs)

model_0_max.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
model_0_max_history=model_0_max.fit(train_data_10_percent,
                                   epochs=5,
                                   steps_per_epoch=len(train_data_10_percent),
                                   validation_data=test_data_10_percent,
                                   validation_steps=len(test_data_10_percent),
                                   callbacks=[create_tensorboard_callback('transfer_learning','feature_extract_maxPool')])

In [44]:
plot_loss_curves(model_0_max_history)

In [45]:
input_shape=(1,5,5,3)
tf.random.set_seed(42)
input_tensor=tf.random.normal(input_shape)
print(input_tensor)

global_avg_pooling=tf.keras.layers.GlobalMaxPool2D()(input_tensor)

print(global_avg_pooling)

print(global_avg_pooling.shape)

print(input_tensor.shape)

try 1% Model with data aug feture extr
10% model with aug feature ext

1% model with aug fine tuning
10% model with aug fine tuning



In [46]:
import zipfile

!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_1_percent.zip
unzip_data("10_food_classes_1_percent.zip")

In [47]:
train_dir_1_percent = "10_food_classes_1_percent/train/"
test_dir = "10_food_classes_1_percent/test/"
IMAGE_SHAPE=(224,224)

In [48]:
train_data_1_percent=tf.keras.preprocessing.image_dataset_from_directory(train_dir_1_percent,
                                                                        batch_size=32,
                                                                        image_size=IMAGE_SHAPE,
                                                                        label_mode='categorical')
test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                             batch_size=32,
                                                             label_mode='categorical',
                                                             image_size=IMAGE_SHAPE)

In [49]:
#augumentation
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

augumented_model=tf.keras.Sequential([
    preprocessing.RandomFlip('horizontal'),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomZoom(0.2)
],name='data_augumentation')

In [50]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
import random

target_class=random.choice(train_data_1_percent.class_names)
target_dir="10_food_classes_1_percent/train/" + target_class
random_target=random.choice(os.listdir(target_dir))
random_image_path=target_dir+'/'+random_target
img=mpimg.imread(random_image_path)

plt.imshow(img)
plt.title('original_image')
plt.axis(False)
aug_image=augumented_model(tf.expand_dims(img,axis=0))
plt.figure()


plt.imshow(tf.squeeze(aug_image)/255.)
plt.title('aug image')

In [51]:
#feature extarction transfer learning 1% data
base_model_2=tf.keras.applications.EfficientNetB0(include_top=False)
base_model_2.trainable=False

inputs=tf.keras.layers.Input(shape=(224,224,3),name='input_layer')
x=augumented_model(inputs)
x=base_model_2(x,training=False)

x=tf.keras.layers.GlobalAveragePooling2D()(x)

outputs=tf.keras.layers.Dense(10,activation='softmax',name='output_layer')(x)

model_2=tf.keras.Model(inputs,outputs)

model_2.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])
model_2_history=model_2.fit(train_data_1_percent,
                           epochs=5,
                           validation_data=test_data,
                           validation_steps=int(0.25*len(test_data)),
                           steps_per_epoch=len(train_data_1_percent),
                           callbacks=[create_tensorboard_callback('transfer_learning','1_percent_data')])

In [52]:
model_2.summary(0)

In [53]:
plot_loss_curves(model_2_history)

In [54]:
model_2.evaluate(test_data)

In [55]:
train_dir = "10_food_classes_10_percent/train/"
test_dir = "10_food_classes_10_percent/test/"

IMAGE_SHAPE=(224,224)

In [56]:
# feature extraction with 10% trainignn data


# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                            label_mode="categorical",
                                                                            image_size=IMG_SIZE)
# Note: the test data is the same as the previous experiment, we could
# skip creating this, but we'll leave this here to practice.
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE)

In [57]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Build data augmentation layer
data_augmentation = Sequential([
  preprocessing.RandomFlip('horizontal'),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomRotation(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNet                 
], name="data_augmentation")

# Setup the input shape to our model
input_shape = (224, 224, 3)

# Create a frozen base model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create input and output layers
inputs = layers.Input(shape=input_shape, name="input_layer") # create input layer
x = data_augmentation(inputs) # augment our training images
x = base_model(x, training=False) # pass augmented images to base model but keep it in inference mode, so batchnorm layers don't get updated: https://keras.io/guides/transfer_learning/#build-a-model 
x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = layers.Dense(10, activation="softmax", name="output_layer")(x)
model_3 = tf.keras.Model(inputs, outputs)

# Compile
model_3.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # use Adam optimizer with base learning rate
              metrics=["accuracy"])

In [58]:
#check point call back
check_point_path='ten_percent_model_checkpoints_weights/checkpoint.ckpt'
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=check_point_path,
                                                      save_best_only=False,
                                                      save_weights_only=True,
                                                      save_freq='epoch',
                                                      verbose=1)

In [59]:
model_3_history=model_3.fit(train_data_10_percent,
                            epochs=5,
                            steps_per_epoch=len(train_data_10_percent),
                            validation_data=test_data,
                            validation_steps=len(test_data),
                            callbacks=[create_tensorboard_callback('transfer_learning','efficient_net_10%_feature'),checkpoint_callback]
                           )

In [60]:
res_model_3=model_3.evaluate(test_data)

In [61]:
plot_loss_curves(model_3_history)

In [62]:
model_4_history=model_3.fit(train_data_10_percent,
                            epochs=10,
                            steps_per_epoch=len(train_data_10_percent),
                            validation_data=test_data,
                            validation_steps=len(test_data),
                            callbacks=[create_tensorboard_callback('transfer_learning','efficient_net_10%_feature'),checkpoint_callback]
                           )

In [63]:
model_3.load_weights(check_point_path)
loaded_weights_model_results=model_2.evaluate(test_data)

In [64]:
res_model_3==loaded_weights_model_results


In [65]:
np.isclose(np.array(loaded_weights_model_results),np.array(res_model_3))

In [66]:
np.array(loaded_weights_model_results)-np.array(res_model_3)

In [67]:
model_3.layers

In [68]:
# create model with fine tuning


base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = True

# ?[layers.trainable for layers in base_model.layers]
for layers in base_model.layers[:-10]:
    layers.trainable=False
    
model_3.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # lr is 10x lower than before for fine-tuning
              metrics=["accuracy"])

In [69]:
[(layers.name,layers.trainable) for layers in base_model.layers]

In [70]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Build data augmentation layer
data_augmentation = Sequential([
  preprocessing.RandomFlip('horizontal'),
  preprocessing.RandomHeight(0.2),
  preprocessing.RandomWidth(0.2),
  preprocessing.RandomZoom(0.2),
  preprocessing.RandomRotation(0.2),
  # preprocessing.Rescaling(1./255) # keep for ResNet50V2, remove for EfficientNet                 
], name="data_augmentation")

# Setup the input shape to our model
input_shape = (224, 224, 3)

# Create a frozen base model
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create input and output layers
inputs = layers.Input(shape=input_shape, name="input_layer") # create input layer
x = data_augmentation(inputs) # augment our training images
x = base_model(x, training=False) # pass augmented images to base model but keep it in inference mode, so batchnorm layers don't get updated: https://keras.io/guides/transfer_learning/#build-a-model 
x = layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = layers.Dense(10, activation="softmax", name="output_layer")(x)
model_2 = tf.keras.Model(inputs, outputs)

# Compile
model_2.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr=0.001), # use Adam optimizer with base learning rate
              metrics=["accuracy"])

In [71]:
initial_epochs = 5
history_10_percent_data_aug = model_2.fit(train_data_10_percent,
                                          epochs=initial_epochs,
                                          validation_data=test_data,
                                          validation_steps=int(0.25 * len(test_data)), # do less steps per validation (quicker)
                                          callbacks=[create_tensorboard_callback("transfer_learning", "10_percent_data_aug"), 
                                                     checkpoint_callback])

In [72]:
base_model.trainable = True

# Freeze all layers except for the
for layer in base_model.layers[:-10]:
    layer.trainable = False

# Recompile the model (always recompile after any adjustments to a model)
model_2.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr=0.0001), # lr is 10x lower than before for fine-tuning
              metrics=["accuracy"])

In [73]:
print(len(model_2.trainable_variables))

In [74]:
fine_tune_epochs = initial_epochs + 5
history_fine_10_percent_data_aug = model_2.fit(train_data_10_percent,
                                               epochs=fine_tune_epochs,
                                               validation_data=test_data,
                                               initial_epoch=history_10_percent_data_aug.epoch[-1], # start from previous last epoch
                                               validation_steps=int(0.25 * len(test_data)),
                                               callbacks=[create_tensorboard_callback("transfer_learning", "10_percent_fine_tune_last_10")])

In [75]:
results_fine_tune_10_percent =model_2.evaluate(test_data)

In [76]:
def compare_history(original_history,new_history,initial_epochs=5):
    """
    Compares two model history objects.
    """
    # Get original history measurements
    acc = original_history.history["accuracy"]
    loss = original_history.history["loss"]

    print(len(acc))

    val_acc = original_history.history["val_accuracy"]
    val_loss = original_history.history["val_loss"]

    # Combine original history with new history
    total_acc = acc + new_history.history["accuracy"]
    total_loss = loss + new_history.history["loss"]

    total_val_acc = val_acc + new_history.history["val_accuracy"]
    total_val_loss = val_loss + new_history.history["val_loss"]

    print(len(total_acc))
    print(total_acc)
    
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(total_acc, label='Training Accuracy')
    plt.plot(total_val_acc, label='Validation Accuracy')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    plt.subplot(2, 1, 2)
    plt.plot(total_loss, label='Training Loss')
    plt.plot(total_val_loss, label='Validation Loss')
    plt.plot([initial_epochs-1, initial_epochs-1],
              plt.ylim(), label='Start Fine Tuning') # reshift plot around epochs
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

In [77]:
compare_history(history_10_percent_data_aug,history_fine_10_percent_data_aug)

In [78]:
#try with all data
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip 
unzip_data("10_food_classes_all_data.zip")

# Setup data directories
train_dir = "10_food_classes_all_data/train/"
test_dir = "10_food_classes_all_data/test/"

In [79]:
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_10_classes_full = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                 label_mode="categorical",
                                                                                 image_size=IMG_SIZE)

# Note: this is the same test dataset we've been using for the previous modelling experiments
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE)

In [80]:
model_2.load_weights(check_point_path)

In [81]:
model_2.evaluate(test_data)

In [82]:

for layer_number, layer in enumerate(model_2.layers):
    print(layer_number, layer.name, layer.trainable)

In [83]:
# Compile
model_2.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # divide learning rate by 10 for fine-tuning
                metrics=["accuracy"])

In [84]:
fine_tune_epochs = initial_epochs + 5

history_fine_10_classes_full = model_2.fit(train_data_10_classes_full,
                                           epochs=fine_tune_epochs,
                                           initial_epoch=history_10_percent_data_aug.epoch[-1],
                                           validation_data=test_data,
                                           validation_steps=int(0.25 * len(test_data)),
                                           callbacks=[create_tensorboard_callback("transfer_learning", "full_10_classes_fine_tune_last_10")])

In [85]:

results_fine_tune_full_data = model_2.evaluate(test_data)
results_fine_tune_full_data

In [87]:

compare_history(original_history=history_10_percent_data_aug,
                 new_history=history_fine_10_classes_full,
                 initial_epochs=5)


In [88]:
!tensorboard dev upload --logdir ./transfer_learning \
  --name "Transfer learning experiments" \
  --description "A series of different transfer learning experiments with varying amounts of data and fine-tuning" \
  --one_shot

In [89]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip 

unzip_data("101_food_classes_10_percent.zip")

train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [90]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)
train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)
                                                                                
test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False) 

In [91]:
checkpoint_path= "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      save_weights_only=True,
                                                      monitor='val_accuracy')

In [92]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
  preprocessing.RandomFlip("horizontal"), # randomly flip images on horizontal edge
  preprocessing.RandomRotation(0.2), # randomly rotate images by a specific amount
  preprocessing.RandomHeight(0.2), # randomly adjust the height of an image by a specific amount
  preprocessing.RandomWidth(0.2), # randomly adjust the width of an image by a specific amount
  preprocessing.RandomZoom(0.2), # randomly zoom into an image
  # preprocessing.Rescaling(1./255) # keep for models like ResNet50V2, remove for EfficientNet
], name="data_augmentation")

In [93]:
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Setup model architecture with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer") # shape of input image
x = data_augmentation(inputs) # augment images (only happens during training)
x = base_model(x, training=False) # put the base model in inference mode so we can use it to extract features without updating the weights
x = layers.GlobalAveragePooling2D(name="global_average_pooling")(x) # pool the outputs of the base model
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x) # same number of outputs as classes
model = tf.keras.Model(inputs, outputs)

In [94]:
model.summary()

In [95]:
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(), # use Adam with default settings
              metrics=["accuracy"])
history_1=model.fit(train_data_all_10_percent,
                   epochs=5,
                   validation_data=test_data,
                   validation_steps=int(0.25*len(test_data)),
                   callbacks=[checkpoint_callback])

In [96]:
# Evaluate model 
results_feature_extraction_model = model.evaluate(test_data)
results_feature_extraction_model

In [97]:
plot_loss_curves(history_1)

In [98]:
base_model.trainable=True
for layers in base_model.layers[:-5]:
    layers.trainable=False

In [99]:
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4), # 10x lower learning rate than default
              metrics=['accuracy'])

In [100]:

fine_tune_epochs = 10
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs=fine_tune_epochs,
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15 * len(test_data)), 
                                                     initial_epoch=history_1.epoch[-1])

In [101]:
results_all_classes_10_percent_fine_tune = model.evaluate(test_data)
results_all_classes_10_percent_fine_tune

In [102]:
compare_history(original_history=history_1,
                 new_history=history_all_classes_10_percent_fine_tune,
                 initial_epochs=5)

In [103]:
model.save("101_food_class_10_percent_saved_big_dog_model")

In [104]:
load_model = tf.keras.models.load_model('101_food_class_10_percent_saved_big_dog_model')

In [105]:
loaded_loss, loaded_accuracy = load_model.evaluate(test_data)
loaded_loss, loaded_accuracy

In [106]:
# Make predictions with model
pred_probs = model.predict(test_data, verbose=1) 

In [107]:
pred_probs.shape

In [108]:
pred_probs[0].argmax(axis=0)

In [109]:
pred_classes = pred_probs.argmax(axis=1)

In [110]:
y_labels=[]
for images,label in test_data.unbatch():
    y_labels.append(label.numpy().argmax())
y_labels[:10]

In [111]:
# Get accuracy score by comparing predicted classes to ground truth labels
from sklearn.metrics import accuracy_score
sklearn_accuracy = accuracy_score(y_labels, pred_classes)
sklearn_accuracy 

In [112]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
plt.figure(figsize=(12,12))
sns.heatmap(confusion_matrix(y_labels, pred_classes))

In [113]:
fig, ax = plt.subplots(figsize=(10,10))
sns.heatmap(confusion_matrix(y_labels, pred_classes))

In [114]:

def load_and_prep_image(filename, img_shape=224, scale=True):
    """
    Reads in an image from filename, turns it into a tensor and reshapes into
    (224, 224, 3).

    Parameters
    ----------
    filename (str): string filename of target image
    img_shape (int): size to resize target image to, default 224
    scale (bool): whether to scale pixel values to range(0, 1), default True
    """
    # Read in the image
    img = tf.io.read_file(filename)
    # Decode it into a tensor
    img = tf.io.decode_image(img)
    # Resize the image
    img = tf.image.resize(img, [img_shape, img_shape])
    if scale:
    # Rescale the image (get all values between 0 and 1)
        return img/255.
    else:
        return img

In [115]:
import os
import random
class_names = test_data.class_names
plt.figure(figsize=(17, 10))
for i in range(3):
  # Choose a random image from a random class 
    class_name = random.choice(class_names)
    filename = random.choice(os.listdir(test_dir + "/" + class_name))
    filepath = test_dir + class_name + "/" + filename

    # Load the image and make predictions
    img = load_and_prep_image(filepath, scale=False) # don't scale images for EfficientNet predictions
    pred_prob = model.predict(tf.expand_dims(img, axis=0)) # model accepts tensors of shape [None, 224, 224, 3]
    pred_class = class_names[pred_prob.argmax()] # find the predicted class 

    # Plot the image(s)
    plt.subplot(1, 3, i+1)
    plt.imshow(img/255.)
    if class_name == pred_class: # Change the color of text based on whether prediction is right or wrong
        title_color = "g"
    else:
        title_color = "r"
    plt.title(f"actual: {class_name}, pred: {pred_class}, prob: {pred_prob.max():.2f}", c=title_color)
    plt.axis(False);

* **BIG daata full data**

In [116]:
import tensorflow_datasets as tfds


In [117]:
datasets_list=tfds.list_builders()
'food101' in datasets_list

In [118]:
(train_data,test_data),ds_info=tfds.load('food101',
                                        split=['train','validation'],
                                        shuffle_files=True,
                                        as_supervised=True,
                                        with_info=True)


In [119]:
ds_info.features

In [120]:
class_names =ds_info.features['label'].names
class_names[:10]

In [121]:
train_data_one_sample=train_data.take(1)

In [122]:
train_data_one_sample

In [123]:
for image,label in train_data_one_sample:
    print(image.shape)
    print(image.dtype)
    print(label)
    print(class_names[label.numpy()])

In [124]:
image

In [125]:
tf.reduce_min(image),tf.reduce_max(image)

In [126]:
plt.imshow(image)

In [127]:
def preprocess_img(image,label,image_size=224):
    img=tf.image.resize(image,[image_size,image_size])
    return tf.cast(img,tf.float32), label

In [128]:
preprocessed_img = preprocess_img(image, label)[0]
preprocessed_img.shape
preprocessed_img

In [129]:
plt.imshow(image)